# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235224325734                   -0.50    8.0   11.0ms
  2   -7.250342837936       -1.82       -1.41    1.0   5.31ms
  3   -7.251206639503       -3.06       -2.00    2.0   30.2ms
  4   -7.251074141510   +   -3.88       -1.95    1.0   5.63ms
  5   -7.251333651395       -3.59       -2.84    1.0   5.47ms
  6   -7.251338305558       -5.33       -3.42    1.0   5.41ms
  7   -7.251338747038       -6.36       -3.71    2.0   6.49ms
  8   -7.251338792088       -7.35       -4.13    1.0   5.53ms
  9   -7.251338797696       -8.25       -4.71    2.0   6.21ms
 10   -7.251338798515       -9.09       -4.93    2.0   6.74ms
 11   -7.251338798687       -9.77       -5.47    1.0   5.60ms
 12   -7.251338798702      -10.81       -5.95    2.0   6.60ms
 13   -7.251338798704      -11.67       -6.72    1.0   5.72ms
 14   -7.251338798705      -12.79       -6.72    3.0   7.63ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.035592272043670894
[ Info: Arnoldi iteration step 2: normres = 0.6293176025539942
[ Info: Arnoldi iteration step 3: normres = 0.8656301896293287
[ Info: Arnoldi iteration step 4: normres = 0.34450852868879184
[ Info: Arnoldi iteration step 5: normres = 0.3271122071851418
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.42e-02, 5.84e-02, 2.45e-01, 2.06e-01, 1.02e-02)
[ Info: Arnoldi iteration step 6: normres = 0.49757220295013643
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.44e-02, 1.94e-01, 4.28e-01, 1.22e-01, 1.06e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07053957747772949
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.56e-04, 2.53e-02, 1.25e-02, 3.04e-02, 4.59e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09744186435304994
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.33